# Basic neuron networks with Tensorflow

In [3]:
# # Check for GPU
# import torch
# print(torch.__version__)
# print(torch.cuda.is_available())
# print(torch.cuda.get_device_name(0))

In [4]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from lab_utils_common import dlc
from lab_coffee_utils import load_coffee_data, plt_roast, plt_prob, plt_layer, plt_network, plt_output_unit
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)


2025-12-16 16:17:28.499380: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-16 16:17:28.499787: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-16 16:17:28.559215: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-16 16:17:30.055462: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

## Implement efficient with `vectorization` and `matmul()`

### Implement efficiently: NO

- Using loop and 1D array

In [15]:
X = np.array([200, 17])
W = np.array([[1, -3, 5], 
              [-2, 4, -6]])
B = np.array([-1, 1, 2])

In [16]:
def g(z):
    return 1 / (1 + np.exp(-z))

In [17]:
def dense(a_in, W, B):
    units = W.shape[1]  # nums of cols in W <---> nums unit in a layer
    a_out = np.zeros(units) # init a zeros array [0, 0, 0,...]
    for cols in range(units):
        z = np.dot(W[:, cols], a_in) + B[cols]
        a_out[cols] = g(z)
    return a_out

In [18]:
a_out = dense(X, W, B)

# Show the result (rounded to 4 decimal places)
print("NO, a_out: ", np.round(a_out, 4))

NO, a_out:  [1. 0. 1.]


### Implement efficiently: YES

- Using: `vectorization` & `matmul()`

In [19]:
# Define all paras is 2D vector -> using matmul() to implement efficiently
X = np.array([[200, 17]])
W = np.array([[1, -3, 5], 
              [-2, 4, -6]])
B = np.array([[-1, 1, 2]])

In [20]:

def dense(a_in, W, B):
    # Parallel compute with vectorization -> using GPUs
    z = np.matmul(a_in, W) + B      # matmul: matrix multiplication 
    print("z: ", z)
    a_out = g(z)
    return a_out

In [21]:
a_out = dense(X, W, B)

# Show the result (rounded to 4 decimal places)
print("YES, a_out: ", np.round(a_out, 4))

z:  [[ 165 -531  900]]
YES, a_out:  [[1. 0. 1.]]


---

---

In [ ]:
x = np.array([[200.0, 17.0],
              [120.0, 5.0],
              [425.0, 20.0],
              [212.0, 18.0]])
y = np.array([1, 0, 0, 1])


In [31]:


#--- Define the model architecture: has 3 layers
# layer 1: 25 neurons, sigmoid activation
# layer 2: 15 neurons, sigmoid activation
# layer 3: 1 neuron, sigmoid activation 
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=25, activation='sigmoid'),
    tf.keras.layers.Dense(units=15, activation='sigmoid'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# model = tf.Sequential([
#     tf.layers.Dense(units=25, activation='sigmoid', input_shape=(2,)),
#     tf.layers.Dense(units=15, activation='sigmoid'),
#     tf.layers.Dense(units=1, activation='sigmoid')
# ])

In [58]:
#--- compile(): configures the model for training
# optimizer: algorithm to use to update weights
# loss: function to minimize
# metrics: list of metrics to monitor during training
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print("Model compiled.")

#--- fit(): trains the model for a fixed number of epochs (iterations on a dataset)
# x: input data
# y: target data
# epochs: number of iterations
# verbose: verbosity mode
model.fit(x, y, epochs=10, verbose=0)
print("Model trained.")

Model compiled.
Model trained.


In [59]:
testing_data = np.array([[201.0, 1]])

In [61]:
#--- predict(): generates output predictions for the input samples
# predictions = model.predict(x)
predictions = model.predict(testing_data)
print(f"- Pred: {predictions}\n")

if predictions < 0.5: 
    print("Oh no!!")
else:
    print("Good coffee!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
- Pred: [[0.48]]

Oh no!!


# The end.